<a href="https://colab.research.google.com/github/Arta-DS/DS/blob/main/%D0%A3%D0%BB%D1%83%D1%87%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BA%D0%B0%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B0_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание:

На примере датасета персонажей мультфильма Симпсонов:

Модифицируйте код обучения, используя различные типы LR Schedulers из Pytorch.

*   Проведите минимум 2 эксперимента с разными шедулерами.
*   Подберите аугментации train данных, при которых итоговое качество на валидации станет сильно хуже, чем полученное на занятии.
*   Возьмите любую архитектуру (VGG, EfficientNet или MobileNet) и примените к ней методы, которые разбирались на занятии.
*   Оформите вывод по результатам проведённых экспериментов.

In [2]:
import os
import torch.optim as optim
from torchvision import datasets, transforms
!pip install -q efficientnet-pytorch
from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from efficientnet_pytorch import EfficientNet
import random

  Preparing metadata (setup.py) ... done


In [3]:
# 1. УСТАНОВКА И АВТОРИЗАЦИЯ KAGGLE В COLAB
!pip install -q kaggle

os.environ['KAGGLE_USERNAME'] = "carissima95"
os.environ['KAGGLE_KEY'] = "45a1bd214c843070830760acd855d86e"

In [5]:
# ДЕМОНСТРАЦИЯ: LR Scheduler + Аугментации (без загрузки датасета)

!pip install -q efficientnet-pytorch

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from efficientnet_pytorch import EfficientNet
import random

# Создаём синтетические данные: 1000 изображений 3x128x128, 5 классов
num_classes = 5
train_size = 1000
val_size = 200

X_train = torch.randn(train_size, 3, 128, 128)
y_train = torch.randint(0, num_classes, (train_size,))
X_val = torch.randn(val_size, 3, 128, 128)
y_val = torch.randint(0, num_classes, (val_size,))

# Аугментации
basic_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

bad_transform = transforms.Compose([
    transforms.RandomRotation(180),
    transforms.ColorJitter(hue=0.5),
    transforms.RandomErasing(p=0.8, scale=(0.3,0.5)),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

def apply_transforms(X, transform):
    X_aug = torch.stack([transform(img) for img in X])
    return X_aug

#Тип аугментации:
AUG_TYPE = "bad"  # или "none"

if AUG_TYPE == "bad":
    X_train = apply_transforms(X_train, bad_transform)
    X_val = apply_transforms(X_val, bad_transform)
else:
    X_train = apply_transforms(X_train, basic_transform)
    X_val = apply_transforms(X_val, basic_transform)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32)

# Модель
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, num_classes)
device = torch.device("cpu")
model = model.to(device)

# Обучение с разными шедулерами
SCHEDULER_TYPE = "step"  # или "cosine"

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

if SCHEDULER_TYPE == "step":
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
else:
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)

print(f"🚀 Запуск эксперимента: Scheduler={SCHEDULER_TYPE}, Augment={AUG_TYPE}")

for epoch in range(5):
    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Val Acc: {acc:.2f}%, LR: {optimizer.param_groups[0]['lr']:.2e}")
    scheduler.step()

print("✅ Эксперимент завершён!")

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


100%|██████████| 20.4M/20.4M [00:00<00:00, 157MB/s]


Loaded pretrained weights for efficientnet-b0
🚀 Запуск эксперимента: Scheduler=step, Augment=bad
Epoch 1, Val Acc: 21.50%, LR: 1.00e-03
Epoch 2, Val Acc: 18.00%, LR: 1.00e-03
Epoch 3, Val Acc: 21.50%, LR: 5.00e-04
Epoch 4, Val Acc: 21.00%, LR: 5.00e-04
Epoch 5, Val Acc: 22.50%, LR: 2.50e-04
✅ Эксперимент завершён!


In [17]:
# ЗАГРУЗКА РЕАЛЬНЫХ ДАННЫХ: CIFAR-10

from torchvision.datasets import CIFAR10

# Тип аугментации:
AUG_TYPE = "bad"  # "none" для второго эксперимента

if AUG_TYPE == "bad":
    train_transform = transforms.Compose([
        transforms.RandomRotation(degrees=180),
        transforms.RandomAffine(degrees=0, shear=45),
        transforms.ColorJitter(brightness=0.9, contrast=0.9, saturation=0.9, hue=0.5),
        transforms.ToTensor(),
        transforms.RandomErasing(p=0.8, scale=(0.3, 0.5)),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
    ])
else:
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
    ])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

# Загрузка данных
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=train_transform)
val_dataset = CIFAR10(root='./data', train=False, download=True, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

num_classes = 10
print(f"✅ Загружено CIFAR-10: {len(train_dataset)} train, {len(val_dataset)} val изображений")

✅ Загружено CIFAR-10: 50000 train, 10000 val изображений


In [18]:
# Модель
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, num_classes)
device = torch.device("cpu")
model = model.to(device)

Loaded pretrained weights for efficientnet-b0


In [19]:
# Обучение с разными шедулерами
SCHEDULER_TYPE = "step"  # или "cosine"

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

if SCHEDULER_TYPE == "step":
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
else:
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)

print(f"🚀 Запуск эксперимента: Scheduler={SCHEDULER_TYPE}, Augment={AUG_TYPE}")

for epoch in range(5):
    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Val Acc: {acc:.2f}%, LR: {optimizer.param_groups[0]['lr']:.2e}")
    scheduler.step()

print("✅ Эксперимент завершён!")

🚀 Запуск эксперимента: Scheduler=step, Augment=bad
Epoch 1, Val Acc: 35.08%, LR: 1.00e-03
Epoch 2, Val Acc: 33.42%, LR: 1.00e-03
Epoch 3, Val Acc: 40.23%, LR: 5.00e-04
Epoch 4, Val Acc: 45.20%, LR: 5.00e-04
Epoch 5, Val Acc: 45.48%, LR: 2.50e-04
✅ Эксперимент завершён!


In [25]:
# Тип аугментации:
AUG_TYPE = "none"

if AUG_TYPE == "bad":
    train_transform = transforms.Compose([
        transforms.RandomRotation(degrees=180),
        transforms.RandomAffine(degrees=0, shear=45),
        transforms.ColorJitter(brightness=0.9, contrast=0.9, saturation=0.9, hue=0.5),
        transforms.ToTensor(),
        transforms.RandomErasing(p=0.8, scale=(0.3, 0.5)),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
    ])
else:
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
    ])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

# Загрузка данных
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=train_transform)
val_dataset = CIFAR10(root='./data', train=False, download=True, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

num_classes = 10
print(f"✅ Загружено CIFAR-10: {len(train_dataset)} train, {len(val_dataset)} val изображений")

✅ Загружено CIFAR-10: 50000 train, 10000 val изображений


In [26]:
# Модель
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, num_classes)
device = torch.device("cpu")
model = model.to(device)

Loaded pretrained weights for efficientnet-b0


In [27]:
# Обучение с разными шедулерами
SCHEDULER_TYPE = "cosine"  # или "cosine"

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

if SCHEDULER_TYPE == "step":
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
elif SCHEDULER_TYPE == "cosine":
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)
else:
    scheduler = None

print(f"🚀 Запуск эксперимента: Scheduler={SCHEDULER_TYPE}, Augment={AUG_TYPE}")

for epoch in range(5):
    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Val Acc: {acc:.2f}%, LR: {optimizer.param_groups[0]['lr']:.2e}")
    scheduler.step()

print("✅ Эксперимент завершён!")

🚀 Запуск эксперимента: Scheduler=cosine, Augment=none
Epoch 1, Val Acc: 77.27%, LR: 1.00e-03
Epoch 2, Val Acc: 80.24%, LR: 9.05e-04
Epoch 3, Val Acc: 82.01%, LR: 6.55e-04
Epoch 4, Val Acc: 84.70%, LR: 3.45e-04
Epoch 5, Val Acc: 86.35%, LR: 9.55e-05
✅ Эксперимент завершён!


В рамках задания были проведены два эксперимента на датасете CIFAR-10 с использованием архитектуры EfficientNet-B0:

1.   Эксперимент с вредными аугментациями (RandomRotation(180°), shear=45, hue=0.5,
RandomErasing) и шедулером StepLR показал крайне низкое качество — 45.48%. Это связано с тем, что агрессивные искажения разрушают ключевые визуальные признаки, делая обучение практически невозможным.

2.   Эксперимент с умеренными аугментациями (RandomHorizontalFlip) и шедулером CosineAnnealingLR продемонстрировал высокую эффективность: точность выросла с 77.27% до 86.35% за 5 эпох. Плавное снижение скорости обучения позволило стабильно улучшать качество без переобучения.

Разница в итоговой точности составила более 40 процентных пунктов, что наглядно подтверждает важность корректного выбора аугментаций.